### Genome Comparison with [elasticsearch](https://www.elastic.co) and [altair](https://altair-viz.github.io/)
#### Overview
This visualization shows large scale changes within genomes since their last common ancestor.  Using three genomes -- human, chimp, gorilla from (links go here).

The sequences are each about 3 billion values grouped into chromosomes, and the input is just raw sequence data (below code just strips out everything else from FASTA file, which is probably not going to get that raw sequence, but it seems to be close enough the visualization still makes sense.

There's two passes over the data, the first pass processes the files into bulk load format for elasticsearch.  The second pass samples each location (below is about 1/10th of 1 percent sample), searches for best match in elasticsearch.  In most cases, the best match will be a corresponding location in another species 
Comparing sequences in order they exist in file, all other text removed.


#### Processing the data
The three sequences total about 9 billion values.  These are put into Elasticsearch, then comparison is just taking a sample, and searching for it in the records.

In the example below, the sequence records represent either 100,000 or 1,000,000 values, which turn into elasticsearch indexes with 90,000 or 9,000 records.

Input data is FASTA files downloaded from [NCBI](ftp://ftp.ncbi.nlm.nih.gov/genomes/)

Steps to process files:

- create file with one long ACGT sequence
- break sequence into fixed size chunks
- *process each chunk into a sequence of "words" (smaller character sequences)*
- *process resulting list of words (e.g. eliminate short words)*
- insert word sequence along with species, chromosome, location (chunk) into an Elasticsearch index

Steps to relate species

- sample each chunk, in example below ~1% of each chunk is used as sample
- process sample exactly same way as chunks processed before insertion into elasticsearch
- search for sample (simple Elasticsearch query)
- search results and scores define closely related genome chunks
- *repeat this sequence for the reverse complement of the sample sequence*

The steps that process the sequence execute *before* data is inserted into Elasticsearch, or they can execute *inside* Elasticsearch via elasticsearch [Character Filters, Tokenizers, and Token Filters](https://www.elastic.co/guide/en/elasticsearch/reference/current/analyzer-anatomy.html).

In [ ]:
import altair as alt
import numpy as np
import pandas as pd
from script_tools.df_process import load_df

alt.renderers.enable()
alt.data_transformers.disable_max_rows()

df = pd.read_csv('data/6primate_chromo_relationships.gz')

ordering = { 
    "Gorilla_gorilla": [ '1', '2A', '2B', '3', '4', '5', '6', '7', '8', '9', '10', 
         '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', 
         '21', '22', 'X' ],
    "hg38": [ '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 
         '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', 
         '21', '22', 'X', 'Y' ],
    "Macaca_fascicularis" :[ '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 
         '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', 
         '21', '22', 'X' ],
    "Macaca_mulatta": [ '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 
         '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', 
         '21', '22', 'X', 'Y' ],
    "Pan_troglodytes": [ '1', '2A', '2B', '3', '4', '5', '6', '7', '8', '9', '10', 
         '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', 
         '21', '22', 'X', 'Y' ],
    "Pongo_abelii":  [ '1', '2A', '2B', '3', '4', '5', '6', '7', '8', '9', '10', 
         '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', 
         '21', '22', 'X' ]
           }


In [ ]:
df.sp.unique()

In [ ]:
df[(df['sp'] == 'Pongo_abelii')]['chr'].unique()

In [ ]:
import itertools

def tuplerow(df, sp, chromo, mchr_order):
    return tuple(df[(df['sp'] == sp) & (df['chr'] == chromo) & (df['mchr'] == mchr)].count() for mchr in mchr_order)

def confusion_matrix_result(df, sp, msp):
    data = df[(df['sp'] == sp) & (df['msp'] == msp)]
    sp_chromo_order = ordering[sp]
    msp_chromo_order = ordering[msp]
    return tuple(tuplerow(df, sp, chromo, msp_chromo_order) for chromo in sp_chromo_order)
    
def confusion_matrix_generator(df):
    for sp, msp in itertools.combinations(df['sp'].unique(), 2):
        yield confusion_matrix_result(df, sp, msp)

In [ ]:
len(m)

In [ ]:
m[0]

In [ ]:

sp = 'Gorilla_gorilla'
msp = 'hg38'

sp_labels = ordering[sp]
msp_labels = ordering[msp]

cm_df = df[(df['sp'] == sp) & (df['msp'] == msp)]
my_list = cm_df['chr'].apply(lambda x: x.upper())
cm_df.insert(0, 'hsf', my_list)
cm_df = cm_df.drop('chr', axis=1)
cm_df = cm_df.rename(columns={"hsf": "chr"})

In [ ]:
def vals(cdf, row_labels, col_labels):
    result = np.zeros(len(row_labels))
    for index, row in cdf.iterrows():
        if row['chr'] in row_labels:
            result[row_labels.index(row['chr'])] = row['count']
    return result

    
# creating a DataFrame with rows from sp, columns from msp
# first create a dictionary with keys from msp, these will be the columns
d = { key:vals(cm_df[cm_df['mchr'] == key], sp_labels, msp_labels) for key in msp_labels}
ddf = pd.DataFrame(d)
ddf.index = sp_labels
ddf

In [ ]:
ddf.index = sp_labels
ddf

In [ ]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

In [ ]:
ddf

In [ ]:
sp,msp

In [ ]:
plt.figure(figsize=(13,10))
plt.xlabel(msp)
plt.ylabel(sp)
ax = sns.heatmap(ddf,xticklabels=True, yticklabels=True)

In [ ]:
sns.heatmap?

In [ ]:
cm_df[cm_df['mchr'] == '2']

In [ ]:
df2

In [ ]:
for index, row in df2.iterrows():
    print(msp_labels.index(row['mchr']), index, row['mchr'])

In [ ]:
conf_df

In [ ]:
cm_df.head()

In [ ]:
cm_df = cm_df.drop(['x','ha','hsf'], axis=1)

In [ ]:
type(cm_df)

In [ ]:
cm_df['x'] = [ s.upper() for s in cm_df['chr']]

In [ ]:
chr1_row_df = cm_df[cm_df['chr'] == '1']

In [ ]:
chr1_row_df

In [ ]:
chr1_row_df[["mchr", "count"]]

In [ ]:
for index, row in chr1_row_df.iterrows():
    result[msp_labels.index(row['mchr'])] = row['count']
result

In [ ]:
msp_labels.index('3')

In [ ]:
chr1_row_df[["mchr","count"]]

In [ ]:
alt.Chart(df_1M).mark_bar().encode(
    alt.X("score:Q", title='Elasticsearch scores, 1M records (3 samples per record)', bin=alt.BinParams(step=1000)),
    y=alt.Y('count()'),
)

#### Example #1: storing 1M bp records in elasticsearch, search for 10k of data
Below shows correspondence between chromosome data based on elasticsearch results.  In most cases, the chromosome data corresponds to same chromosome number.  But there are some large scale structural changes that show up in the data.

The relationship between the chromosomes of two species is determined by Elasticsearch, below are a couple ways to see this.  First is by just a dataframe, second is a heatmap.

In [ ]:
hc_relationships = related_species.getChromosomeRelationships('human', 'chimp', 100)
hc_relationships

In [ ]:
# or even more interesting, the chromosomes that have relationships to more than one other
counts = hc_relationships.groupby(['human']).count()
counts = counts[counts['count'] > 1]
hc_relationships[hc_relationships['human'].isin(counts.index)]

In [ ]:
hg_relationships = related_species.getChromosomeRelationships('gorilla', 'human', 10)
counts = hg_relationships.groupby(['gorilla']).count()
counts = counts[counts['count'] > 1]
hg_relationships[hg_relationships['gorilla'].isin(counts.index)]

In [ ]:
cg_relationships = related_species.getChromosomeRelationships('chimp', 'gorilla', 200)
counts = cg_relationships.groupby(['chimp']).count()
counts = counts[counts['count'] > 1]
cg_relationships[cg_relationships['chimp'].isin(counts.index)]

In [ ]:
species_graph.chromosomeRelationships('human', 'chimp')

In [ ]:
species_graph.chromosomeRelationships('human', 'gorilla')

In [ ]:
species_graph.chromosomeRelationships('chimp', 'gorilla')

In [ ]:
domains = [ 'same', 'inversed']
color_scale = alt.Scale(
    domain=domains,
    range=['#6baed6', '#fcae91']
)

def cgraph(df, top_species, top_chromosome, middle_species, middle_chromosome, bottom_species, bottom_chromosome, graph_width=600):
    g = alt.Chart(df).mark_line().encode(
        x=alt.X('x',axis=alt.Axis(grid=False)),
        y=alt.Y('y',axis=alt.Axis(grid=False)),
        x2='x2',
        y2='y2',
        opacity=alt.value(0.3),
        color=alt.Color('orientation:N', title='',scale=color_scale)
    )
    
    maxVals = df.max()
    maxCenter = maxVals['x']
    maxRest = maxVals['x2']
    maxY = max(maxVals['y'], maxVals['y2'])
     
    # here I just want a bar at the top, and text on the right that says:  species, chromosome
    # top_data + bars, gives me a transparent green bar at top, with no text
    top_label = f"{top_species}, {top_chromosome}"
    middle_label = f"{middle_species}, {middle_chromosome}"
    bottom_label = f"{bottom_species}, {bottom_chromosome}"
    X_MARGIN = 10
    Y_MARGIN = 12
    top_data = pd.DataFrame({
        'x': [ maxRest + X_MARGIN, maxRest + X_MARGIN, maxRest + X_MARGIN ],
        'y': [ maxY - Y_MARGIN, int(maxY/2), Y_MARGIN ],
        'text': [ top_label, middle_label, bottom_label ]
    })
    speciesLabels = alt.Chart(top_data).mark_text(
        stroke='grey',
        opacity=0.9, 
        fontSize=10,
        fontStyle="italic",
        align="left"
    ).encode(
        x=alt.X('x:Q'),
        y=alt.Y('y:Q'),
        text=alt.Text('text'),
        color=alt.Color('orientation:N', legend=None, scale=color_scale)
    )
    
    x = alt.Chart().mark_text().encode(
        x=alt.X('x:Q', axis=alt.Axis(title='million bp', grid=False, ticks=True)),
        y=alt.Y('y:Q', axis=alt.Axis(title='', grid=False, labels=False, ticks=False)),
        color=alt.Color('orientation:N', legend=alt.Legend(orient="left",title='', symbolType="stroke"), scale=color_scale)
    )

    return alt.layer(x, speciesLabels, g).configure_view(
        stroke='transparent',
        width=graph_width
    ).configure_axis(grid=False)


def sp_to_y(val, top, mid, bot):
    if val == bot:
        return 0
    elif val == mid:
        return 200
    else:
        return 400

def mod_df(df, top_sp, top_chr, top_start_loc, middle_sp, middle_chr, middle_start_loc, bottom_sp, bottom_chr, bottom_start_loc, min_score=1000):
    """Lines out from middle to top and bottom"""
    df = df[df['sp'] == middle_sp]
    df = df[df['score'] > min_score]
    df = df[df['chr'] == middle_chr]
    df = df[((df['mchr'] == top_chr) & (df['msp'] == top_sp)) | ((df['mchr'] == bottom_chr) & (df['msp'] == bottom_sp))]
    df['x'] = [(x - middle_start_loc)/1000000 for x in df['loc']]
    df['y'] = [ sp_to_y(val, top_sp, middle_sp, bottom_sp) for val in df['sp']]
    df['y2'] = [ sp_to_y(val, top_sp, middle_sp, bottom_sp) for val in df['msp']]
    df['x2'] = [ ((x - top_start_loc)/1000000) if y == 400 else ((x - bottom_start_loc)/1000000) 
                for x,y in zip(df['mloc'], df['y2'])]
    df = df[df['x'] >= 0]
    return df

def graph_df(df, top_sp, top_chr, top_start_loc, middle_sp, middle_chr, bottom_sp, bottom_chr, min_score=1000, graph_width=600):
    df = mod_df(df, top_sp, top_chr, 0, middle_sp, middle_chr, 0, bottom_sp, bottom_chr, 0, min_score)
    return cgraph(df, top_sp, top_chr, middle_sp, middle_chr, bottom_sp, bottom_chr, graph_width)

def chromosome_compare(df, top_spec, middle_spec, bottom_spec, min_score=1000, graph_width=600):
    top_sp, top_chr, top_start_loc = top_spec
    middle_sp, middle_chr, middle_start_loc = middle_spec
    bottom_sp, bottom_chr, bottom_start_loc = bottom_spec
    df = mod_df(df, top_sp, top_chr, top_start_loc, middle_sp, middle_chr, middle_start_loc, bottom_sp, bottom_chr, bottom_start_loc, min_score)
    return cgraph(df, top_sp, top_chr, middle_sp, middle_chr, bottom_sp, bottom_chr, graph_width)

# either refine 3 species graph or make it generic and able to handle same species (for duplicates)
# parameters should be (species, chromosome, start_loc, end_loc)


##### Chromosome 1:  Human + Chimp + Gorilla

When 3 species are shown, we can identify which species had what sort of large scale event (inversion, duplication, splitting, joining).

For example, below there are events like:
- a large inversion in chimp 2A, and a smaller human inversion
- a large section of chimp 7 getting duplicated onto the end of chimp 7 (needs some more investigation)


*Chromosome 1 events*
- Gorilla, chromosome 1, a sequence of inversion events

In [ ]:
mdf = mod_df(df_1M, "human", "1", 0, "chimp", "1", 0, "gorilla", "1", 0)
len(mdf),mdf

In [ ]:
chromosome_compare(TEST_DF, ("human", "1", 0), ("chimp", "1", 0), ("gorilla", "1", 0))


In [ ]:
from species.related_species import StructuralRelationships
sr = StructuralRelationships(TEST_DF, 'chimp', '1', 'gorilla', '1', 100000)
list(sr.similar_sections(2))

*Chromosome 2 events*
- Chimp, large 2A inversion
- Human, smaller 2 inversion

In [ ]:
chromosome_compare(TEST_DF, ("human", "2", 0), ("gorilla", "2A", 0), ("chimp", "2A", 0))

*Chromosome 2 events*
- Human, 2 chromosomes merge into one

below shows right side of Human 2 same as 2B for Chimp and Gorilla.

In [ ]:
chromosome_compare(TEST_DF, ("human", "2", 105000000), ("chimp", "2B", 0), ("gorilla", "2B", 0))

*Chromosome 3 events*
- Human, several large inversions

In [ ]:
chromosome_compare(TEST_DF, ('human', '3', 0), ('chimp', '3', 0), ('gorilla','3', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', '4', 0), ('chimp', '4', 0), ('gorilla', '4', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', '5', 0), ('chimp', '5', 0), ('gorilla', '5', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', '6', 0), ('chimp', '6', 0), ('gorilla', '6', 0))

*Chromosome 7 events*
- looks like several sections got duplicated onto end (unexpected)

In [ ]:
chromosome_compare(TEST_DF, ('human', '7', 0), ('chimp', '7', 0), ('gorilla', '7', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', '8', 0), ('chimp', '8', 0), ('gorilla', '8', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', '9', 0), ('chimp', '9', 0), ('gorilla', '9', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', '10', 0), ('chimp', '10', 0), ('gorilla', '10', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', '11', 0), ('chimp', '11', 0), ('gorilla', '11', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', '12', 0), ('chimp', '12', 0), ('gorilla', '12', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', '13', 0), ('chimp', '13', 0), ('gorilla', '13', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', '14', 0), ('chimp', '14', 0), ('gorilla', '14', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', '15', 0), ('chimp', '15', 0), ('gorilla', '15', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', '16', 0), ('chimp', '16', 0), ('gorilla', '16', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', '17', 0), ('chimp', '17', 0), ('gorilla', '17', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', '18', 0), ('chimp', '18', 0), ('gorilla', '18', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', '19', 0), ('chimp', '19', 0), ('gorilla', '19', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', '20', 0), ('chimp', '20', 0), ('gorilla', '20', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', '21', 0), ('chimp', '21', 0), ('gorilla', '21', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', '22', 0), ('chimp', '22', 0), ('gorilla', '22', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', 'X', 0), ('chimp','X', 0), ('gorilla', 'X', 0))

Large parts of gorilla chromosome 5 correspond to human and chimp chromosome 17.
This graph looks wrong (like there is a bug) -- there is way too much crossing of same orientation lines, in all other graphs these are parallel, and only inverted lines cross in middle.

In [ ]:
chromosome_compare(TEST_DF, ('chimp', '17', 0), ('gorilla', '5', 0), ('human', '17', 0))

In [ ]:
chromosome_compare(TEST_DF, ('human', '17', 0), ('gorilla', '5', 0), ('chimp', '17', 0))

In [ ]:
human_1M = related_species.getSpecies('human')
chimp_1M = related_species.getSpecies('chimp')
gorilla_1M = related_species.getSpecies('gorilla')

In [ ]:
print("hcg sizes: ", len(human_1M), len(chimp_1M), len(gorilla_1M))
hstr = f"{human_1M['score'].min()}-{human_1M['score'].max()}"
cstr = f"{chimp_1M['score'].min()}-{chimp_1M['score'].max()}"
gstr = f"{gorilla_1M['score'].min()}-{gorilla_1M['score'].max()}"
print(f"hcg range: {hstr}, {cstr}, {gstr}")

In [ ]:
chr_map = related_species.chr_order

def ab_count(df, chrVal, mchrVal):
    result = 0
    dataVal = df[(df['chr'] == chrVal) & (df['mchr'] == mchrVal)]
    if len(dataVal) == 1:
        result = dataVal.iloc[0]['count']
    return result

def duplicate_grid(df, sp):
    sp1_chr_list = chr_map[sp]
    sp2_chr_list = chr_map[sp]
    ax1chrs = [ v1 for v1 in sp1_chr_list for v2 in sp2_chr_list]
    ax2chrs = [ v2 for v1 in sp1_chr_list for v2 in sp2_chr_list]
    counts = []
    for ax1_chr,ax2_chr in zip(ax1chrs,ax2chrs):
        count = ab_count(df, ax1_chr, ax2_chr)
        if ax1_chr != ax2_chr:
            count += ab_count(df, ax2_chr, ax1_chr)
        counts.append(count)
    source = pd.DataFrame({'x': ax1chrs,
                           'y': ax2chrs,
                           'z': counts})
    return source

def duplicate_graph(df, sp):
    ax1_title = f"{sp} chromosomes"
    ax2_title = f"{sp} chromosomes"
    return alt.Chart(df).mark_rect().encode(
        x=alt.X('x:N', sort=chr_map[sp], axis=alt.Axis(title=ax1_title, grid=True, ticks=True)),
        y=alt.Y('y:N', sort=chr_map[sp], axis=alt.Axis(title=ax2_title, grid=True, ticks=True)),
        color=alt.Color('z:Q', title="count")
    )

def duplicate_heatmap(df, sp):
    """create heatmap for duplicated chunks"""
    ddg = df.groupby(['chr', 'mchr']).count()
    ddg.reset_index(inplace=True)
    grid = duplicate_grid(ddg, sp)
    return duplicate_graph(grid, sp)

Two unexpected things about the duplicates:
- most duplicates are within a chromosome (that's why they are excluded below, they dominate)
- those NOT in the same chromosome seem to be in X

In [ ]:
duplicate_heatmap(related_species.getSpeciesXchr('human'), 'human')

In [ ]:
duplicate_heatmap(related_species.getSpeciesXchr('chimp'), 'chimp')

In [ ]:
duplicate_heatmap(related_species.getSpeciesXchr('gorilla'), 'gorilla')

In [ ]:

def labels_graph(df, top_label, middle_label, bottom_label):
    domains = [ 'same', 'inversed']
    color_scale = alt.Scale(
        domain=domains,
        range=['#6baed6', '#fcae91']
    )

    # df represented lines are all from the center horizontal line to the top or bottom horizonal line
    lines = alt.Chart(df).mark_line().encode(
        x=alt.X('x',axis=None), #alt.Axis(grid=False)),
        y=alt.Y('y',axis=alt.Axis(grid=False)),
        x2='x2',
        y2='y2',
        color=alt.Color('orientation:N', scale=color_scale)
    )
    
    maxVals = df.max()
    max_Y_value = maxVals['y2']
     
    # make a DataFrame to hold the location of the labels
    X_MARGIN = 10
    Y_MARGIN = 12
    top_data = pd.DataFrame({
        'x': [ X_MARGIN, X_MARGIN, X_MARGIN ],
        'y': [ max_Y_value - Y_MARGIN, int(max_Y_value/2), Y_MARGIN ],
        'text': [ top_label, middle_label, bottom_label ]
    })
    three_stacked_labels = alt.Chart(top_data).mark_text(
        stroke='grey',
        opacity=0.9, 
        fontSize=10,
        fontStyle="italic",
    ).encode(
        x=alt.X('x:Q'),
        y=alt.Y('y:Q'),
        text=alt.Text('text'),
        color=alt.Color('orientation:N', legend=None, scale=color_scale)
    )
    
    orientation_legend = alt.Chart().mark_text().encode(
        x=alt.X('x:Q', axis=alt.Axis(title='A label and axis that I want', grid=False, ticks=True)),
        y=alt.Y('y:Q', axis=None), #alt.Axis(title='A LINE that I do NOT want', grid=False, labels=False, ticks=False)),
        color=alt.Color('orientation:N', legend=alt.Legend(orient="right", title='', symbolType="stroke"), scale=color_scale)
    )

    return alt.layer(orientation_legend, three_stacked_labels, lines).configure_view(
        stroke='transparent'
    ).configure_axis(grid=False)
#     return alt.layer(orientation_legend, lines).configure_view(
#         stroke='transparent'
#     ).configure_axis(grid=False)

# either refine 3 species graph or make it generic and able to handle same species (for duplicates)
# parameters should be (species, chromosome, start_loc, end_loc)

In [ ]:
df = pd.read_csv('graph_example.csv')
labels_graph(df, 'TOP label', 'MIDDLE label', 'BOTTOM label')

In [ ]:
df[:10]
